# Network attention

In this tutorial we will learn how to analyse the attention of a network model. We are going to: 

1) Derive saliency maps

2) Compare saliency maps for different inputs

3) (optional) Derive class activation maps and compare

### Data and trained network model
The data are simulated astrophysical 3D lightcones of the 21cm signal of the forbidden spin-flip transition of neutral hydrogen, generated with the semi-numerical code 21cmFAST (https://github.com/21cmfast/21cmFAST), in the case of mock lightcones noise was generated with the code 21cmSense (https://github.com/jpober/21cmSense), to represent data as expected for the Square Kilometre Array at radio wavelengths. The trained network model is the 3D-21cmPIE-Net (see https://github.com/stef-neu/3D-21cmPIE-Net) as published in Neutsch, Heneka, Brueggen 2022 (arXiv:2201.07587). Credit of the code go to Steffen Neutsch, Caroline Heneka 2022. 

### General remarks
As we use trained models for this task, GPU are not necessarily required.

In [ ]:
# some packages that we will need
#uncomment on google colab:
# !git clone https://github.com/csheneka/introspection-tutorial.git
#!pip install tensorflow==2.9.1
#!pip install tf-keras-vis tensorflow

import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib import cm

In [ ]:
# optional color-scheme for plotting later on
from matplotlib import colors
eor_colour = colors.LinearSegmentedColormap.from_list(
    "EoR",
    [
        (0, "white"),
        (0.21, "yellow"),
        (0.42, "orange"),
        (0.63, "red"),
        (0.86, "black"),
        (0.9, "blue"),
        (1, "cyan"),
    ],
)
plt.register_cmap(cmap=eor_colour)

# Change the sigmoid activation of the last layer to a linear one to not obstruct attention.
def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

## 1) Derive saliency maps

To derive saliency, we need to load both model and data. The trained model is available both trained on simulations-only and on mock data including noise. 

In [ ]:
# Load the trained model(s)
# model 1 simulations-only: models/3D_21cmPIE_Net_sim_par6.h5
# model 2 mock: models/3D_21cmPIE_Net_optmock_par6.h5
model = keras.models.load_model('models/3D_21cmPIE_Net_sim_par6.h5')
#google colab: one might have to adjust the path e.g. to 'introspection-tutorial/models/3D_21cmPIE_Net_sim_par6.h5'

# Read light-cones from tfrecords files
HII_DIM=140
BOX_LEN=2350
nlabel = 6
nz = 92
def parse_function(files):
    keys_to_features = {"label":tf.io.FixedLenFeature((nlabel),tf.float32),
                        "image":tf.io.FixedLenFeature((),tf.string),
                        "tau":tf.io.FixedLenFeature((),tf.float32),
                        "gxH":tf.io.FixedLenFeature((nz),tf.float32),
                        "z":tf.io.FixedLenFeature((nz),tf.float32),}
    parsed_features = tf.io.parse_example(files, keys_to_features)
    image = tf.io.decode_raw(parsed_features["image"],tf.float16)
    image = tf.reshape(image,(HII_DIM,HII_DIM,BOX_LEN))
    return image, parsed_features["label"] # Image, label ordering: m_WDM,Omega_m,L_X,E_0,T_vir,zeta

data_file="data/lc0.tfrecord"  # larger file: lc1n.tfrecord
#google colab: one might have to adjust the path e.g. to 'introspection-tutorial/data/lc0.tfrecord'
dataset = tf.data.TFRecordDataset(data_file)
dataset = dataset.map(parse_function,num_parallel_calls=tf.data.experimental.AUTOTUNE)
data = np.array(list(dataset.as_numpy_iterator()),dtype=object)
print(data.shape)
print(data[0,0].shape)

We now move on to derive saliency, using the tf-keras-vis package.

### Task 1: 
Explore plotting saliency maps, how do they look, what do they tell?

### Question 1: 
What do you notice, which areas are spatially and temporally important?

In [ ]:
from tf_keras_vis.saliency import Saliency

# create saliency object
obj_saliency = Saliency(model, model_modifier=model_modifier,clone=True) 

# generate saliency maps for parameters i.e. classes
parameters=[0,1,2,3,4,5]
lc = data[0,0]
for para in parameters:
    def loss(output):
        return output[0][para] # shape (samples,classes)
    map_saliency = obj_saliency(loss,lc.reshape(?,?,?,1))

print(map_saliency.shape)
nslice = 
plt.figure()
plot_saliency = plt.imshow(map_saliency[?], cmap=cm.hot)
plt.figure()
plot_lc = plt.imshow(lc[nslice,:,:], cmap="EoR",vmin=-150,vmax=30)

## 2a) Compare saliency maps for different inputs

### Task 2a: 
Plot both saliency for simulations-only and mock lightcones

### Question 2a: 
Does the network's attention shift with the inclusion of noise? If yes, how?

In [ ]:
# Task 2, plot and compare here

## 2b) Beyond vanilla saliency 
### Question 2b: 
Can the mapping of attention with saliency be improved, in particular can noise in the maps be reduced? Tipp: Use SmoothGrad from tf-keras-vis. What does SmoothGrad exploit?

### Task 2b:
Explore SmoothGrad for different setting (smooth_samples, smooth_noise).

In [ ]:
#  Question 2b, try out SmoothGrad here
from tf_keras_vis.saliency import Saliency

# Change the sigmoid activation of the last layer to a linear one to not obstruct attention.
def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

# create saliency object
obj_saliency = Saliency(?) 

nsamples = 
noise = 

# generate saliency maps for parameters i.e. classes
parameters=[0,1,2,3,4,5]
lc = data[0,0]
for para in parameters:
    def loss(output):
        return output[0][para] # shape (samples,classes)
    map_saliency = obj_saliency(?(see above),?(see above),smooth_samples=nsamples,smooth_noise=noise)

print(map_saliency.shape)
nslice =
plt.figure()
plot_saliency = plt.imshow(map_saliency[?], cmap=cm.hot)
plt.figure()
plot_lc = plt.imshow(lc[nslice,:,:], cmap="EoR",vmin=-150,vmax=30)

## 3) (optional) Derive class activation maps and compare 

### Task 3: 
Create CAMs, explore their spatial and temporal structure. 

In [ ]:
# note: 
# there is currently an incompatibility of CAM generated with tf-keras-vis and the newest tensorflow version 2.9
# will need to downgrade tensorflow version
from tf_keras_vis.gradcam import Gradcam

# create CAM object
obj_cam = gradcam.Gradcam(model, model_modifier=model_modifier,clone=True) 

# generate heatmaps for attention
parameters=[0,1,2,3,4,5]
lc = data[0,0]
for para in parameters:
    def loss(output):
        return output[0][para] # shape (samples,classes)
    map_cam = obj_cam(loss,lc.reshape(140,140,2350,1),penultimate_layer=-1) # focus on last convolutional layer

print(map_cam.shape)
nslice = 130
plt.figure()
plot_cam = plt.imshow(map_cam[0,nslice,:,:], cmap=cm.hot)
plt.figure()
plot_lc = plt.imshow(lc[nslice,:,:], cmap="EoR",vmin=-150,vmax=30)